In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [24]:
#J82JWhLzSODJvD9bnMurt7EV35121qTaI65ci6Ph
# Open the desired year of nsrdb data
# server endpoint, username, password is found via a config file
df = pd.read_csv ('./dataset/1060699_25.77_-80.18_2019.csv', skiprows=[0,1])
df.info()

df_header = pd.read_csv ('./dataset/1060699_25.77_-80.18_2019.csv',nrows=1)
df_header.info()
pd.set_option('display.max_columns', None)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Year                                         17520 non-null  int64  
 1   Month                                        17520 non-null  int64  
 2   Day                                          17520 non-null  int64  
 3   Hour                                         17520 non-null  int64  
 4   Minute                                       17520 non-null  int64  
 5   DHI                                          17520 non-null  int64  
 6   DNI                                          17520 non-null  int64  
 7   GHI                                          17520 non-null  int64  
 8   Clearsky DHI                                 17520 non-null  int64  
 9   Clearsky DNI                                 17520 non-null  int64  
 10

In [25]:
df_header.head()

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Clearsky DHI Units,Clearsky DNI Units,Clearsky GHI Units,Dew Point Units,DHI Units,DNI Units,GHI Units,Solar Zenith Angle Units,Temperature Units,Pressure Units,Relative Humidity Units,Precipitable Water Units,Wind Direction Units,Wind Speed,Cloud Type -15,Cloud Type 0,Cloud Type 1,Cloud Type 2,Cloud Type 3,Cloud Type 4,Cloud Type 5,Cloud Type 6,Cloud Type 7,Cloud Type 8,Cloud Type 9,Cloud Type 10,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,1060699,-,-,-,25.77,-80.18,-5,0,-5,w/m2,w/m2,w/m2,c,w/m2,w/m2,w/m2,Degree,c,mbar,%,cm,Degrees,m/s,NaN,Clear,Probably Clear,Fog,Water,Super-Cooled Water,Mixed,Opaque Ice,Cirrus,Overlapping,Overshooting,Unknown,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.1.1


In [26]:
df.head()

,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Solar Zenith Angle,Fill Flag,Surface Albedo,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm),Unnamed: 24
0,2019,1,1,0,0,0,0,0,0,0,0,0,21.8,173.88,0,0.08,3.7,2.4,128,87.10,24.1,1019,0.0,0.0,NaN
1,2019,1,1,0,30,0,0,0,0,0,0,0,21.8,176.95,0,0.08,3.6,2.4,128,87.10,24.1,1019,0.0,0.0,NaN
2,2019,1,1,1,0,0,0,0,0,0,0,0,21.7,171.38,0,0.08,3.6,2.4,127,86.53,24.1,1019,0.0,0.0,NaN
3,2019,1,1,1,30,0,0,0,0,0,0,0,21.7,164.76,0,0.08,3.5,2.4,126,86.53,24.1,1019,0.0,0.0,NaN
4,2019,1,1,2,0,0,0,0,0,0,0,0,21.5,158.02,0,0.08,3.5,2.4,126,85.94,24.0,1019,0.0,0.0,NaN


In [33]:
df = df[(df['Hour'] < 22) & (df['Hour'] > 5)]
#df.head()

In [28]:
df_x = df[['Surface Albedo','Cloud Type', 
    'Dew Point', 'Solar Zenith Angle', 'Pressure', 'Wind Direction', 'Wind Speed', 
    'Relative Humidity', 'Temperature', 'Precipitable Water']]
df_x.head()

,Surface Albedo,Cloud Type,Dew Point,Solar Zenith Angle,Pressure,Wind Direction,Wind Speed,Relative Humidity,Temperature,Precipitable Water
0,0.08,0,21.8,173.88,1019,128,3.7,87.10,24.1,2.4
1,0.08,0,21.8,176.95,1019,128,3.6,87.10,24.1,2.4
2,0.08,0,21.7,171.38,1019,127,3.6,86.53,24.1,2.4
3,0.08,0,21.7,164.76,1019,126,3.5,86.53,24.1,2.4
4,0.08,0,21.5,158.02,1019,126,3.5,85.94,24.0,2.4


In [29]:
df_y= df[['GHI']]

In [34]:
df_x.shape

,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Solar Zenith Angle,Fill Flag,Surface Albedo,Wind Speed,Precipitable Water,Wind Direction,Relative Humidity,Temperature,Pressure,Global Horizontal UV Irradiance (280-400nm),Global Horizontal UV Irradiance (295-385nm),Unnamed: 24
12,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
13,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
14,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
15,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
16,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17511,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
17512,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
17513,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
17514,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [31]:
model = LinearRegression()
scores = []
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for i, (train, test) in enumerate(kfold.split(df_x, df_y)):
 model.fit(df_x.iloc[train,:], df_y.iloc[train,:])
 score = model.score(df_x.iloc[test,:], df_y.iloc[test,:])
 scores.append(score)
print(scores)

[0.726591086164976, 0.741054161077946, 0.7424673231732196, 0.7324217136951283, 0.7521021352305346, 0.7277205838974465, 0.7349491519275929, 0.7348873525617305, 0.7336884164093279, 0.7287275071806367]


In [32]:
pca = PCA()

pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])
    



NameError: name 'logistic' is not defined